In [13]:
import cv2
import numpy as np
import tensorflow as tf
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

def build_generator():
    """ Define a simple CNN generator for watermark prediction """
    inputs = tf.keras.layers.Input(shape=(128, 128, 1))
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = tf.keras.Model(inputs, x, name="Generator")
    return model

generator = build_generator()
weights_path = "./model_weights/generator_epoch_4900.weights.h5"

if os.path.exists(weights_path):
    generator.load_weights(weights_path)
    print("✅ Generator weights loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Weights file not found at {weights_path}")

INPUT_VIDEO_PATH = "./input1.mp4"
OUTPUT_VIDEO_PATH = "./output_watermarked1.mp4"
GENERATED_WATERMARK_PATH = "./generated_watermarks"
EXTRACTED_WATERMARK_PATH = "./extracted_watermarks"

os.makedirs(GENERATED_WATERMARK_PATH, exist_ok=True)
os.makedirs(EXTRACTED_WATERMARK_PATH, exist_ok=True)

cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

frame_count = 0
psnr_values = []
ssim_values = []
watermark_ssim_values = []
watermark_mse_values = []

alpha = 0.07

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  

    frame = frame.astype(np.float32) / 255.0  

    gray_frame = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (128, 128)) / 255.0  

    input_frame = np.expand_dims(resized_frame, axis=[0, -1])
    watermark = generator.predict(input_frame)[0, :, :, 0]  

    watermark_resized = cv2.resize(watermark, (frame_width, frame_height))

    generated_watermark_path = os.path.join(GENERATED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(generated_watermark_path, (watermark_resized * 255).astype(np.uint8))

    watermarked_frame = frame + alpha * np.expand_dims(watermark_resized, axis=-1)
    watermarked_frame = np.clip(watermarked_frame * 255, 0, 255).astype(np.uint8)

    out.write(watermarked_frame)

    original_frame = (frame * 255).astype(np.uint8)
    psnr_value = psnr(original_frame, watermarked_frame, data_range=255)
    ssim_value = ssim(original_frame, watermarked_frame, data_range=255, win_size=3, channel_axis=-1)

    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)

    extracted_watermark = (watermarked_frame.astype(np.float32) - frame * 255) / alpha
    extracted_watermark = np.mean(extracted_watermark, axis=-1) 

    extracted_watermark = cv2.GaussianBlur(extracted_watermark, (3, 3), 0)

    extracted_watermark = np.clip(extracted_watermark, 0, 255).astype(np.uint8)
    
    extracted_watermark_path = os.path.join(EXTRACTED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(extracted_watermark_path, extracted_watermark)

    h, w = watermark_resized.shape
    h_start, h_end = h // 4, 3 * h // 4
    w_start, w_end = w // 4, 3 * w // 4
    cropped_original = watermark_resized[h_start:h_end, w_start:w_end]
    cropped_extracted = extracted_watermark[h_start:h_end, w_start:w_end]

    watermark_ssim = ssim(cropped_original, cropped_extracted.astype(np.float32) / 255, data_range=1.0)
    watermark_mse = mean_squared_error(cropped_original.flatten(), cropped_extracted.flatten() / 255)

    watermark_ssim_values.append(watermark_ssim)
    watermark_mse_values.append(watermark_mse)

    frame_count += 1

cap.release()
out.release()

avg_watermark_ssim = np.mean(watermark_ssim_values)
avg_watermark_mse = np.mean(watermark_mse_values)

print(f"✅ Watermarked video saved: {OUTPUT_VIDEO_PATH}")
print(f"✅ Generated watermarks saved in: {GENERATED_WATERMARK_PATH}")
print(f"✅ Extracted watermarks saved in: {EXTRACTED_WATERMARK_PATH}")
print(f"📊 Watermark SSIM: {avg_watermark_ssim:.4f} (Closer to 1 is better)")
print(f"📊 Watermark MSE: {avg_watermark_mse:.6f} (Lower is better)")
print(f"🎯 Model Accuracy (Based on SSIM): {avg_watermark_ssim * 100:.2f}%")


✅ Generator weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━